# ThaiPy Bangkok

# Functions are objects


In [1]:
def my_function(name):
    print("Hello {}".format(name))

In [2]:
my_function

<function __main__.my_function>

In [3]:
my_function(my_function)

Hello <function my_function at 0x10c9b5510>


## A function in a function (closure)

In [25]:
def greeter(greet):
    def action(name):
        print("{} {}".format(greet, name))
    return action

my_greeter = greeter("Hello")
my_greeter("people")
my_greeter("python")

Hello people
Hello python


In [5]:
hellower = greeter("Hello")
hellower

<function __main__.greeter.<locals>.action>

In [6]:
dearer = greeter("Dear")
dearer("people")
hellower("people")

Dear people
Hello people


# Wrapping functions aka decorators

In [7]:
def rainbows(original):
    def wrapped():
        print("🌈🌈🌈")
        original()
    return wrapped

@rainbows
def hello():
    print("Hello")
    
hello()

🌈🌈🌈
Hello


## Decorator with a parameter

In [8]:
def rainbows(amount):
    def decorator(original):
        def wrapped():
            print("🌈" * amount)
            original()
        return wrapped
    return decorator

@rainbows(12)
def hello():
    print("Hello")
    
hello()

🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈
Hello


# The not so nice things

## Original function arguments

In [9]:
def rainbows(amount):
    def decorator(original):
        def wrapped(*args, **kwds):
            print("🌈" * amount)
            return original(*args, **kwds)
        return wrapped
    return decorator

@rainbows(12)
def hello(name):
    """Docstring"""
    print("Hello {}".format(name))
    
hello("people")

🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈
Hello people


## Original function information

In [10]:
print(hello.__name__)
print(hello.__doc__)

wrapped
None


In [11]:
from functools import wraps

def rainbows(amount):
    def decorator(original):
        @wraps(original)
        def wrapped(*args, **kwds):
            print("🌈" * amount)
            return original(*args, **kwds)
        return wrapped
    return decorator

@rainbows(12)
def hello(name):
    """Docstring"""
    print("Hello {}".format(name))
    
print(hello.__name__)
print(hello.__doc__)

hello
Docstring


# The bad

## Function signature

In [12]:
def baddecorator(original):
    def wrapped(prefix, name):
        original(prefix + name)
    return wrapped

@baddecorator
def hello(name):
    print("Hello {}".format(name))
    
hello("people")

TypeError: wrapped() missing 1 required positional argument: 'name'

## Hidden dependencies

In [13]:
import requests

def baddecorator2(original):
    def wrapped(name):
        requests.get('https://google.com',params={"q": name})
        original(name)
    return wrapped

@baddecorator2
def hello(name):
    print("Hello {}".format(name))
    
hello("people")

Hello people


## Namespace WTF

In [14]:
def badbaddecorator(original):
    return 42

@badbaddecorator
class Hello:
    pass

print(Hello)

42


# Code as configuration
## Venusian - A library for deferring decorator actions

In [15]:
def my_action():
    return {"name": "Python"}

res = my_action()
print(res, type(res))

{'name': 'Python'} <class 'dict'>


In [16]:
import json

def json_output(original):
    def wrapped():
        result = original()
        return json.dumps(result)
    return wrapped

@json_output
def my_action():
    return {"name": "Python"}

res = my_action()
print(res, type(res))

{"name": "Python"} <class 'str'>


In [17]:
import venusian

def json_output(original):
    def callback(scanner, name, ob):
        def wrapped():
            result = original()
            return json.dumps(result)
        scanner.registry.append((name, wrapped))
    venusian.attach(original, callback)
    return original

@json_output
def my_action():
    return {"name": "Python"}

res = my_action()
print(res, type(res))

{'name': 'Python'} <class 'dict'>


In [18]:
import sys

registry = []
scanner = venusian.Scanner(registry=registry)
scanner.scan(sys.modules[__name__])

registry

[('my_action',
  <function __main__.json_output.<locals>.callback.<locals>.wrapped>)]

In [19]:
res = registry[0][1]()
print(res, type(res))

{"name": "Python"} <class 'str'>


## Dectate - A configuration engine for Python frameworks


In [20]:
import dectate

class MyFilters(dectate.App):
    pass

In [21]:
@MyFilters.directive('filter')
class FilterActions(dectate.Action):
    config = {
       'filters': dict
    }
    def __init__(self, name):
        self.name = name

    def identifier(self, filters):
        return self.name

    def perform(self, obj, filters):
        filters[self.name] = obj

In [22]:
@MyFilters.filter('json')
def to_json():
    pass 

@MyFilters.filter('yaml')
def to_yaml():
    pass

dectate.commit(MyFilters)

print(MyFilters.config.filters)

{'yaml': <function to_yaml at 0x10cc241e0>, 'json': <function to_json at 0x10cc24bf8>}


In [23]:
@MyFilters.filter('json')
def to_json_super():
    pass

dectate.commit(MyFilters)

ConflictError: Conflict between:
  File "<ipython-input-22-8caa4ff29457>", line 1
    def to_json():
  File "<ipython-input-23-1af76039f7f4>", line 1
    def to_json_super():